# Backtest Avancé du Modèle Morningstar (GA+CNN+LSTM)

In [ ]:
from Morningstar.workflows.morningstar_trading import MorningstarTradingWorkflow
from Morningstar.model.architecture.morningstar_model import MorningstarTradingModel
from Morningstar.configs.morningstar_config import MorningstarConfig, TradingStyle, RiskLevel
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, HTML
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configuration dynamique
config = MorningstarConfig(
    timeframe='1h',
    trading_style=TradingStyle.SWING,
    risk_level=RiskLevel.MODERE,
    cnn_filters=64,
    lstm_units=64
)

# Chargement du modèle avec vérification
try:
    model = MorningstarTradingModel(input_shape=(60, 19), num_classes=3)
    model.model = tf.keras.models.load_model('morningstar_model.h5')
    print("✅ Modèle chargé avec succès")
except Exception as e:
    print(f"❌ Erreur de chargement: {e}")

# Initialisation avec gestion de risque dynamique
trader = MorningstarTradingWorkflow(
    model=model,
    config=config,
    initial_capital=10000,
    dynamic_risk=True
)

# Chargement des données avec vérification d'intégrité
data = pd.read_parquet('data/BTCUSDT_1h.parquet')
print(f"📊 Données chargées: {len(data)} points")

# Backtest avec suivi des performances
print("⚡ Lancement du backtest...")
await trader.run_backtest(data)

# Analyse avancée
results = trader.analyzer.analyze(trader.trade_history)

# Dashboard interactif
display(HTML(f"""
<h2>📈 Performance du Modèle</h2>
<table style='width:100%'>
  <tr><td>Return:</td><td style='color:{"green" if results["return"] > 0 else "red"}'>{results['return']:.2f}%</td></tr>
  <tr><td>Sharpe Ratio:</td><td>{results['sharpe']:.2f}</td></tr>
  <tr><td>Max Drawdown:</td><td style='color:red'>{results['max_drawdown']:.2f}%</td></tr>
  <tr><td>Win Rate:</td><td>{results['win_rate']:.2f}%</td></tr>
  <tr><td>Profit Factor:</td><td>{results['profit_factor']:.2f}</td></tr>
</table>
"""))

# Visualisations améliorées
plt.figure(figsize=(14, 10))

# 1. Courbe de capital avec drawdown
plt.subplot(3, 1, 1)
plt.plot(results['equity_curve'], label='Capital')
plt.fill_between(range(len(results['drawdown'])), 
                 results['equity_curve'], 
                 results['equity_curve'] - (results['equity_curve'] * results['drawdown']/100),
                 color='red', alpha=0.3, label='Drawdown')
plt.title('Performance du Capital')
plt.legend()

# 2. Heatmap des performances par heure
plt.subplot(3, 1, 2)
hourly_perf = trader.trade_history.groupby(
    pd.to_datetime([t['time'] for t in trader.trade_history]).hour
)['profit'].mean()
sns.heatmap(hourly_perf.to_frame().T, cmap='RdYlGn', annot=True, fmt=".1f")
plt.title('Performance par Heure de la Journée')

# 3. Distribution des gains/pertes
plt.subplot(3, 1, 3)
profits = [t['profit'] for t in trader.trade_history if t['profit'] != 0]
sns.histplot(profits, kde=True, bins=30)
plt.axvline(0, color='red', linestyle='--')
plt.title('Distribution des Résultats par Trade')

plt.tight_layout()
plt.show()

# Analyse des signaux avec contexte
signals = pd.DataFrame({
    'time': [t['time'] for t in trader.trade_history],
    'price': [t['price'] for t in trader.trade_history],
    'type': [t['type'] for t in trader.trade_history],
    'profit': [t['profit'] for t in trader.trade_history],
    'confidence': [t['confidence'] for t in trader.trade_history]
})

plt.figure(figsize=(14, 6))
plt.plot(data['close'].tail(500), label='Prix', alpha=0.5)
plt.scatter(signals.index, 
            signals['price'][signals['type'] == 'buy'], 
            c=signals['confidence'][signals['type'] == 'buy'],
            cmap='Greens', label='Achat', s=100)
plt.scatter(signals.index, 
            signals['price'][signals['type'] == 'sell'], 
            c=signals['confidence'][signals['type'] == 'sell'],
            cmap='Reds', label='Vente', s=100)
plt.colorbar(label='Confiance du Modèle')
plt.title('Signaux de Trading avec Niveau de Confiance')
plt.legend()
plt.show()

# Export des résultats
signals.to_csv('backtest_results.csv', index=False)
print("💾 Résultats sauvegardés dans backtest_results.csv")